In [ ]:
import pandas as pd
import requests
from datetime import datetime, date
import json

In [ ]:
# API Request for Data
url = "https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com"
cm_endpoint = "/api/1/StudyHack/cm/subject/list"
ae_endpoint = "/api/1/StudyHack/ae/subject/list"

# CM Data
resp = requests.get(url+cm_endpoint)
response = resp.json()
cm_data = response.get('data', [])


# AE Data
resp = requests.get(url+ae_endpoint)
response = resp.json()
ae_data = response.get('data', [])


In [ ]:
err_results = []

In [ ]:
post_endpoint = "/api/1/StudyHack/query"
headers = {'Content-type': 'application/json', 'Accept': 'application/json'}

In [ ]:
# Type3
for subject_id in ae_data:
    if subject_id in cm_data:
        cm_endpoint = "/api/1/StudyHack/cm/subject/%s/list"%(subject_id)
        ae_endpoint = "/api/1/StudyHack/ae/subject/%s/list"%(subject_id)
        cm_resp = requests.get(url+cm_endpoint)
        ae_resp = requests.get(url+ae_endpoint)
        cm_ddata = cm_resp.json()
        ae_ddata = ae_resp.json()
        # CM & AE Data
        df1 = pd.json_normalize(cm_ddata['data'])
        df2 = pd.json_normalize(ae_ddata['data'])
        # ae & cm duplicates/overlap
        prev_end_date = None
        prev_st_date = None
        prev_formidx = None
        for i in range(df2.shape[0]):
            prev_values_bool = prev_st_date and prev_end_date and prev_formidx
            if df2.iloc[i]['cmstdat_dts'] and df2.iloc[i]['cmendat_dts']:    
                try:
                    date1 = (datetime.strptime(df2.iloc[i]['cmstdat_dts'],'%m/%d/%Y')).date()
                    date2 = (datetime.strptime(df2.iloc[i]['cmendat_dts'],'%m/%d/%Y')).date()
                    if i!=0 and prev_values_bool and (date1 <= prev_end_date and date1 >= prev_st_date):
                        query_dict = {
                          "email_address": "amrs.tech@gmail.com",
                          "formname": str(df2.iloc[i]['formname']),
                          "formid": str(df2.iloc[i]['formid']),
                          "formidx": str(df2.iloc[i]['formidx']),
                          "type": "TYPE3",
                          "subjectid": str(df2.iloc[i]['subjectid'])
                        }
                        resp = requests.post(url+post_endpoint, data=json.dumps(query_dict), headers=headers)
                        err_results.append((query_dict,resp.json()))
                        print(resp.json())
                    if date1 and date2 and df2.iloc[i]['formidx']:  
                        prev_end_date = date2
                        prev_st_date = date1
                        prev_formidx = df2.iloc[i]['formidx']
                except Exception as e:
                    print(e)
# Type4
        prev_end_date = None
        prev_st_date = None
        prev_formidx = None
        for i in range(df1.shape[0]):
            prev_values_bool = prev_st_date and prev_end_date and prev_formidx
            if df1.iloc[i]['cmstdat_dts'] and df1.iloc[i]['cmendat_dts']:    
                try:
                    date1 = (datetime.strptime(df1.iloc[i]['cmstdat_dts'],'%m/%d/%Y')).date()
                    date2 = (datetime.strptime(df1.iloc[i]['cmendat_dts'],'%m/%d/%Y')).date()
                    if i!=0 and prev_values_bool and (date1 <= prev_end_date and date1 >= prev_st_date):
                        query_dict = {
                          "email_address": "amrs.tech@gmail.com",
                          "formname": str(df1.iloc[i]['formname']),
                          "formid": str(df1.iloc[i]['formid']),
                          "formidx": str(df1.iloc[i]['formidx']),
                          "type": "TYPE4",
                          "subjectid": str(df1.iloc[i]['subjectid'])
                        }
                        resp = requests.post(url+post_endpoint, data=json.dumps(query_dict), headers=headers)
                        err_results.append((query_dict,resp.json()))
                        print(resp.json())
                    if date1 and date2 and df1.iloc[i]['formidx']:  
                        prev_end_date = date2
                        prev_st_date = date1
                        prev_formidx = df1.iloc[i]['formidx']
                except Exception as e:
                    print(e)
    

